In [1]:
!pip install -q accelerate -U
!pip install -q transformers
!pip install -q datasets
!pip install -q jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.9 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

from datasets import load_dataset, Dataset
import soundfile as sf
from jiwer import wer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from transformers import AutoProcessor, WhisperForConditionalGeneration

processor = AutoProcessor.from_pretrained("byoussef/whisper-large-v2-Ko")
model = WhisperForConditionalGeneration.from_pretrained("byoussef/whisper-large-v2-Ko").to('cuda')

In [8]:
from tqdm.notebook import tqdm
debug_file = open("debug.txt","w")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
wav_path = '/content/drive/MyDrive/의현/감정 분류를 위한 대화 음성 데이터셋/4차년도.zip'
!mkdir ./wav/
!cp -r "$wav_path" ./wav/
drive.flush_and_unmount()

In [8]:
from IPython.display import clear_output

!unzip './wav/4차년도.zip' -d file/

clear_output()

In [4]:
def get_files(folder):
    file_dict = {}
    cnt = 0
    for root, dirs, files in os.walk(folder):
        for filename in files:
            if cnt >= 100:
                break
            cnt += 1
            if not filename.endswith('.wav'):
                continue
            filepath = os.path.join(root,filename)
            file_dict[filename] = filepath
    return file_dict

In [5]:
def transcribe(signal):
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            signal = torch.from_numpy(signal)
            inputs = processor(signal, sampling_rate = 16000, return_tensors="pt").to(device)
            input_features = inputs.input_features
            generated_ids = model.generate(inputs=input_features)
            transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return transcription

In [6]:
def load_audio(raw_folder,trans_folder):
    files = get_files(raw_folder)
    for file,file_path in tqdm(files.items()):
        trans_path_0 = os.path.join(trans_folder,file.replace('.wav','0.txt'))
        trans_path_1 = os.path.join(trans_folder,file.replace('.wav','1.txt'))
        if os.path.exists(trans_path_0):
            continue
        audio_input, _ = librosa.load(file_path, sr=16000,mono=False)
        if audio_input.ndim == 1:
            trans_0 = transcribe(audio_input)
            with open(trans_path_0, 'w', encoding = 'utf-8') as f:
                f.write(trans_0)
        else:
            channel_0 = audio_input[0]
            channel_1 = audio_input[1]
            trans_0 = transcribe(channel_0)
            trans_1 = transcribe(channel_1)
            with open(trans_path_0, 'w', encoding = 'utf-8') as f:
                f.write(trans_0)
            with open(trans_path_1, 'w', encoding = 'utf-8') as f:
                f.write(trans_1)

In [9]:
if __name__ == "__main__":
    !mkdir ./transcript_4
    raw_folder = "./file"
    trans_folder = "./transcript_4"
    load_audio(raw_folder,trans_folder)

mkdir: cannot create directory ‘./transcript_4’: File exists


  0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:5                                                                              │
│ in load_audio:10                                                                                 │
│ in transcribe:7                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/whisper/modeling_whisper.py:1663 in  │
│ generate                                                                                         │
│                                                                                                  │
│   1660 │   │   if generation_config.return_timestamps:                                           │
│   1661 │   │   │   logits_processor = [WhisperTimeStampLogitsProcessor(generation_config)]       │
│   1662 │   │                                                                                     │
│ ❱ 1663 │   │   return super().generate(                                                          │
│   1664 │   │   │   inputs,                                                                       │
│   1665 │   │   │   generation_config,                                                            │
│   1666 │   │   │   logits_processor,                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1522 in generate        │
│                                                                                                  │
│   1519 │   │   │   │   )                                                                         │
│   1520 │   │   │                                                                                 │
│   1521 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1522 │   │   │   return self.greedy_search(                                                    │
│   1523 │   │   │   │   input_ids,                                                                │
│   1524 │   │   │   │   logits_processor=logits_processor,                                        │
│   1525 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2339 in greedy_search   │
│                                                                                                  │
│   2336 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2337 │   │   │                                           

In [10]:
import os
import codecs

txt_files = [f for f in os.listdir(trans_folder) if f.endswith('.txt')]

txt_files = txt_files[:10]

for txt_file in txt_files:
    with codecs.open(os.path.join(trans_folder, txt_file), 'r', encoding='utf8') as file:
        print(f"File: {txt_file}")
        print(file.read())
        print("\n----------------\n")

File: 5e37a3d433e9ad176cc9afd50.txt
어 해봤는데 진지하게도 말해봤는데 안 고쳐지더라고 얘는 좀 짜증나네

----------------

File: 5e350ce25807b852d9e074930.txt
없어 걔가 나 싫은가 봐

----------------

File: 5e37ef777995ef170fc0f59d0.txt
친구 생일이라 좀 어려울 것 같아

----------------

File: 5e369155dbc4b7182a6a94470.txt
고마워 네가 나한테 많은 도움이 됐어

----------------

File: 5e392f9033e9ad176cc9ba930.txt
와 진짜 대박이다 이거 누가 토해 놓은 거지

----------------

File: 5e33b22f5807b852d9e052110.txt
간혹 싸우기는 했는데 자주 그러지는 않았어

----------------

File: 5e49c70bc38c123b9ec6dba00.txt
방금 가봤더니 그대로 있어

----------------

File: 5e5205808849ac3aacea56560.txt
화냈는데도 습관을 못 고쳐

----------------

File: 5e468f6c2a2d173b73a02b520.txt
고등학교 동창인데 이렇게 방을 더럽게 쓸 줄 몰랐어

----------------

File: 5e3671f5c8c25f16cd144c810.txt
으 누가 화장실을 토하고 안 씻겠어

----------------

